In [1]:
import sympy as sym
sym.init_printing()
#https://www.youtube.com/watch?v=_xomX-d8XU4 пример отсюда(https://www.youtube.com/watch?v=_xomX-d8XU4)

Схема взята с 
[youtube:How to Solve a Kirchhoff''s Rules Problem - Matrix Example](https://www.youtube.com/watch?v=_xomX-d8XU4)
Далее по тексту, как используя SymPy, производить вычисления в матричном виде.
Собственно схема:

![title](img/circuit1.png)

### Матрица соединений $A$
Матрица соединений $A=[a_{ij}]$  имеет ненулевые значения:
*   -1, когда ток $I_i$  течет в направлении из вершины $x_j$ 
*   1, когда ток $I_i$ течет в направлении к вершине $x_j$

Строки соответствуют ребрам (токам $I_i$), столбцы - вершинам $x_j$ 

Например, для тока $I_1$ из вершины $x_1$ в направлении к вершине $x_2$ строка матрицы будет $[-1,1,0,0]$

In [2]:
# Зададим токи как символьные переменные в SymPy
I1,I2,I3,I4,I5,I6 = sym.symbols("I1 I2 I3 I4 I5 I6")
I = sym.Matrix([I1,I2,I3,I4,I5,I6])

In [3]:
A = sym.Matrix([[-1,1,0,0],[0,-1,0,1],[0,0,1,-1],[1,0,-1,0],[1,0,-1,0],[0,-1,0,1]])
A

⎡-1  1   0   0 ⎤
⎢              ⎥
⎢0   -1  0   1 ⎥
⎢              ⎥
⎢0   0   1   -1⎥
⎢              ⎥
⎢1   0   -1  0 ⎥
⎢              ⎥
⎢1   0   -1  0 ⎥
⎢              ⎥
⎣0   -1  0   1 ⎦

### Матрицы контуров $B$ и сопротивлений $R$
Матрица контуров $B=[b_{ij}]$  имеет ненулевые значения:
*   1, когда направление тока $I_i$ совпадает с направлением обхода контура $B_i$ 
*   -1, когда когда направление тока $I_i$ противоположно направлению обхода контура $B_i$ 

Строки соответствуют контурам ($B_1,B_2,B_3$), столбцы - токам $I_1...I_6$ 

Например, для контура $B_1$ в контором присутствуют токи $I_6,I_2$ строка матрицы будет $[0,0,0,-1,1,0]$

In [4]:
B = sym.Matrix([[0,-1,0,0,0,1],[1,1,1,1,0,0],[0,0,0,-1,1,0]])
B

⎡0  -1  0  0   0  1⎤
⎢                  ⎥
⎢1  1   1  1   0  0⎥
⎢                  ⎥
⎣0  0   0  -1  1  0⎦

Отразим сопротивления на участках, через которые протекают токи $I_1..I_6$ в виде диагональной матрицы

In [5]:
R=sym.Matrix.diag(2,5,0,6,8,10)
R

⎡2  0  0  0  0  0 ⎤
⎢                 ⎥
⎢0  5  0  0  0  0 ⎥
⎢                 ⎥
⎢0  0  0  0  0  0 ⎥
⎢                 ⎥
⎢0  0  0  6  0  0 ⎥
⎢                 ⎥
⎢0  0  0  0  8  0 ⎥
⎢                 ⎥
⎣0  0  0  0  0  10⎦

### Матрица ЭДС в контурах $E$
Для каждого контура $B_1...B_3$ подсчитаем в них участвующую ЭДС (с учетом полярности по направлению обхода):

In [6]:
E = sym.Matrix([-8,4,2])
E

⎡-8⎤
⎢  ⎥
⎢4 ⎥
⎢  ⎥
⎣2 ⎦

### 1 правило Кирхгофа (сумма токов в узлах равна нулю)
Используем его (решив уравнение $A^T\times I=0$):

In [7]:
sym.solve(A.T*I)

Обновим информацию по токам согласно полученному решению (как сделать непосредственно через функции SymPy - не нашел)

In [8]:
I = sym.Matrix([I4+I5, I4+I5-I6, I4+I5, I4, I5, I6])
I

⎡  I₄ + I₅   ⎤
⎢            ⎥
⎢I₄ + I₅ - I₆⎥
⎢            ⎥
⎢  I₄ + I₅   ⎥
⎢            ⎥
⎢     I₄     ⎥
⎢            ⎥
⎢     I₅     ⎥
⎢            ⎥
⎣     I₆     ⎦

### 2 правило Кирхгофа (сумма произведений тока на сопротивления в закрытом контуре есть сумма ЭДС в нем)
Используем его(решив уравнение $B\times R \times I=E$)

In [9]:
sym.solve(B*R*I-E)

Подсчитаем в числовом виде, найденные значения токов $I_1...I_6$ на участках:

In [10]:
I_calc=I.evalf(subs=sym.solve(B*R*I-E))
I_calc

⎡0.25 ⎤
⎢     ⎥
⎢ 0.7 ⎥
⎢     ⎥
⎢0.25 ⎥
⎢     ⎥
⎢  0  ⎥
⎢     ⎥
⎢0.25 ⎥
⎢     ⎥
⎣-0.45⎦

### Подсчитаем разность потенциалов и мощности на резисторах
Выпишем в матрицу: 
*  1 столбец - сопротивления всех резисторов из нашей схемы 
*  2 столбец - номера контуров $B_1...B_3$, в состав которых они входят 
*  3 столбец - номера текущих через них токов $I_1...I_6$

![title](img/circuit1.png)

In [11]:
R_circuit=sym.Matrix([[5,3,5],[3,3,5],[6,3,4],[2,2,1],[5,2,2],[2,1,6],[8,1,6]])
R_circuit

⎡5  3  5⎤
⎢       ⎥
⎢3  3  5⎥
⎢       ⎥
⎢6  3  4⎥
⎢       ⎥
⎢2  2  1⎥
⎢       ⎥
⎢5  2  2⎥
⎢       ⎥
⎢2  1  6⎥
⎢       ⎥
⎣8  1  6⎦

Для удобства добавим числовое значение тока и числовое значение ЭДС в контуре, при расчетах учитываем что индексация в SymPy начинается с нуля, поэтому вычитаем 1 при нумерации элементов

In [12]:
R_circuit=R_circuit.col_insert(3,R_circuit.col(2).applyfunc(lambda i: I_calc[i -1]))
R_circuit=R_circuit.col_insert(4,R_circuit.col(1).applyfunc(lambda i: E[i -1]))
R_circuit

⎡5  3  5  0.25   2 ⎤
⎢                  ⎥
⎢3  3  5  0.25   2 ⎥
⎢                  ⎥
⎢6  3  4    0    2 ⎥
⎢                  ⎥
⎢2  2  1  0.25   4 ⎥
⎢                  ⎥
⎢5  2  2   0.7   4 ⎥
⎢                  ⎥
⎢2  1  6  -0.45  -8⎥
⎢                  ⎥
⎣8  1  6  -0.45  -8⎦

Тогда разность потенциалов будет $U=I \cdot R$, добавим ее в конце отдельной колонкой.

In [13]:
R_circuit=R_circuit.col_insert(5,R_circuit.col(0).multiply_elementwise(R_circuit.col(3)))
R_circuit

⎡5  3  5  0.25   2   1.25⎤
⎢                        ⎥
⎢3  3  5  0.25   2   0.75⎥
⎢                        ⎥
⎢6  3  4    0    2    0  ⎥
⎢                        ⎥
⎢2  2  1  0.25   4   0.5 ⎥
⎢                        ⎥
⎢5  2  2   0.7   4   3.5 ⎥
⎢                        ⎥
⎢2  1  6  -0.45  -8  -0.9⎥
⎢                        ⎥
⎣8  1  6  -0.45  -8  -3.6⎦

Добавим последней колонкой мощность $P=U \cdot I$:

In [14]:
R_circuit=R_circuit.col_insert(6,R_circuit.col(5).multiply_elementwise(R_circuit.col(3)))
R_circuit

⎡5  3  5  0.25   2   1.25  0.3125⎤
⎢                                ⎥
⎢3  3  5  0.25   2   0.75  0.1875⎥
⎢                                ⎥
⎢6  3  4    0    2    0      0   ⎥
⎢                                ⎥
⎢2  2  1  0.25   4   0.5   0.125 ⎥
⎢                                ⎥
⎢5  2  2   0.7   4   3.5    2.45 ⎥
⎢                                ⎥
⎢2  1  6  -0.45  -8  -0.9  0.405 ⎥
⎢                                ⎥
⎣8  1  6  -0.45  -8  -3.6   1.62 ⎦

Добавим, для удобства, титульную строку с заголовками.

In [15]:
R_circuit=R_circuit.row_insert(0,sym.Matrix([sym.symbols("R_Ohm B_i I_j EForce_V Current_A Voltage_V Power_Wt")]))
R_circuit

⎡R_Ohm  Bᵢ  I_j  EForce_V  Current_A  Voltage_V  Power_Wt⎤
⎢                                                        ⎥
⎢  5    3    5     0.25        2        1.25      0.3125 ⎥
⎢                                                        ⎥
⎢  3    3    5     0.25        2        0.75      0.1875 ⎥
⎢                                                        ⎥
⎢  6    3    4      0          2          0         0    ⎥
⎢                                                        ⎥
⎢  2    2    1     0.25        4         0.5      0.125  ⎥
⎢                                                        ⎥
⎢  5    2    2     0.7         4         3.5       2.45  ⎥
⎢                                                        ⎥
⎢  2    1    6    -0.45       -8        -0.9      0.405  ⎥
⎢                                                        ⎥
⎣  8    1    6    -0.45       -8        -3.6       1.62  ⎦